<a href="https://colab.research.google.com/github/DebraBeat/stock_project/blob/main/stock_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, SoupStrainer
import re
import csv
from google.colab import drive
import os
import datetime
import time
import random
import subprocess

In [ ]:
# Downloaded csv file and locally parsed symbols from here: https://www.nasdaq.com/market-activity/stocks/screener
# Code to parse symbols:
# import csv
# path = r"C:\Users\zeeri\Downloads\nasdaq_screener_1717802878001.csv"
# tickers = []

# with open(path, newline='') as csvfile:
#     reader = csv.reader(csvfile, delimiter=',')

#     for row in reader:
#         tickers.append(row[0])

# print(tickers[1:])

# Get symbol data
# Run ls to make sure you're in the right directory

'''
Put ourselves into the google drive directory for our project and get a list
of symbols to use
'''

drive.mount('/content/drive', force_remount=True)
os.chdir("drive/My Drive/stock_project")
!ls
raw_symbols = []
symbols = []
with open('symbols', 'r') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')

  for row in reader:
    raw_symbols.append(row)

# symbols is a 2d list of one element, so make it the first element
raw_symbols = raw_symbols[0]

# sanitize symbols
for symbol in raw_symbols:
  if symbol.isalnum():
    symbols.append(symbol)

print(len(raw_symbols))
print(len(symbols))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-4dfc6d8591a7>", line 24, in <cell line: 24>
    os.chdir("drive/My Drive/stock_project")
OSError: [Errno 107] Transport endpoint is not connected: 'drive/My Drive/stock_project'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of

In [ ]:
'''
Define our user agent so yahoo finance doesn't think we're a web crawler.
Define the key parts of our URL to request
'''
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}

head = 'https://finance.yahoo.com/quote/'
stats_tail = '/key-statistics/'
hist_tail = 'history?period1=942883200&period2=1717718400&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true'

In [ ]:
'''
This cell is a prototype / test of fetching data from yahoo finance, to be used below.
Make sure to run this to create table_elements
'''


# response = requests.get('https://finance.yahoo.com/quote/NVDA/key-statistics/', headers=header)
response = requests.get(head + 'NVDA' + stats_tail, headers=header)
print(head + symbols[0] + stats_tail)
soup = BeautifulSoup(response.text, 'html.parser')
data_cells = soup.find_all('td')
table_elements = []

for tag in data_cells:
  table_elements.append(tag.contents[0])
print(table_elements)

for i, s in enumerate(table_elements):
  try:
    if s[0].isalpha():
      # print(f'{s}: {table_elements[i+1]}')
      pass
  except KeyError as error:
    # print('--')
    pass

https://finance.yahoo.com/quote/A/key-statistics/
['Market Cap', '3.19T ', '2.13T ', '1.52T ', '1.01T ', '1.15T ', '686.27B ', 'Enterprise Value', '3.17T ', '2.11T ', '1.51T ', '1.00T ', '1.15T ', '685.00B ', 'Trailing P/E', '75.84 ', '72.42 ', '81.17 ', '98.50 ', '243.38 ', '159.48 ', 'Forward P/E', '50.00 ', '35.71 ', '30.40 ', '24.51 ', '62.89 ', '61.73 ', 'PEG Ratio (5yr expected)', '1.49 ', '1.16 ', '0.60 ', '0.82 ', '2.66 ', '3.45 ', 'Price/Sales', '40.52 ', '35.37 ', '34.14 ', '31.09 ', '45.06 ', '25.79 ', 'Price/Book', '64.88 ', '49.45 ', '45.57 ', '36.57 ', '47.05 ', '31.05 ', 'Enterprise Value/Revenue', '39.71 ', '34.64 ', '33.63 ', '30.62 ', '44.46 ', '25.39 ', 'Enterprise Value/EBITDA', '62.51 ', '59.31 ', '66.16 ', '77.37 ', '178.68 ', '114.42 ', <div class="plusupsell svelte-10u0d9f" data-testid="upsell-col"><header class="x-small mb-0 svelte-1e5hi02"><h3 class="header tw-text-xl svelte-1e5hi02">Upgrade to begin using 40 years of earnings data and get so much more.</h3> <

In [ ]:
'''
This cell is a prototype / test of creation of the statistics dataframe, to be used below.
'''

cols = ['Company']
d = {'Company': symbols[0]}
for i, s in enumerate(table_elements):
  try:
    if s[0].isalpha():
      cols.append(s)
      d[s] = table_elements[i+1]
  except KeyError:
    cols.append(np.nan)

df = pd.DataFrame(data=d, columns=cols, index=[0])
df.head()

,Company,Market Cap,Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5yr expected),Price/Sales,Price/Book,Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Shares Short (prior month 4/15/2024),Forward Annual Dividend Rate,Forward Annual Dividend Yield,Trailing Annual Dividend Rate,Trailing Annual Dividend Yield,Payout Ratio,Dividend Date,Ex-Dividend Date,Last Split Factor,Last Split Date
0,A,3.19T,3.17T,75.84,50.00,1.49,40.52,64.88,39.71,62.51,...,290.75M,0.04,0.03%,0.02,0.01%,0.94%,6/28/2024,6/11/2024,10:1,6/10/2024


In [ ]:
'''
Define our function for retriveing stock statistics (AKA data or valuations
or metrics). Take in a DataFrame, create a new row for a new stock, and
concatenate it onto the end of the DataFrame. Then return the new DataFrame.
'''

def append_stock_stats(df, symbol, i):
  response = requests.get(head + symbol + stats_tail, headers=header)
  soup = BeautifulSoup(response.text, 'html.parser')
  data_cells = soup.find_all('td')
  table_elements = []
  d = {'Company': symbol}

  for tag in data_cells:
    try:
      table_elements.append(tag.contents[0])
    except IndexError as err:
      table_elements.append(np.nan)
      print(f'{err}')

  for j, s in enumerate(table_elements):
    try:
      if s[0].isalpha() and j < len(table_elements) - 1:
        d[s] = table_elements[j+1]
    except KeyError:
      # print(f'KeyError, Symbol: {symbol}, i: {i}')
      # print(f'j: {j}, s: {s}')
      pass
    except IndexError:
      print(f'IndexError, Symbol: {symbol}, i: {i}, j: {j}')
      print(f'j: {j}, s: {s}')
      raise
    except TypeError:
      print(f'TypeError, Symbol: {symbol}, i: {i}, j: {j}')
      print(f'j: {j}, s: {s}')
      raise
    except requests.TooManyRedirects:
      print(f'TooManyRedirects, Symbol: {symbol}, i: {i}, j: {j}')
      print(f'j: {j}, s: {s}')
      raise

  row = pd.DataFrame(data=d, columns=cols, index=[i])
  df = pd.concat([df, row])

  return df

In [ ]:
'''
Create and populate our DataFrame of stock data.
'''

# Create columns for reference
cols = ['Company']
for s in table_elements:
  try:
    if s[0].isalpha():
      cols.append(s)
  except KeyError:
    cols.append(np.nan)

# Create empty DataFrame with just column names
df = pd.DataFrame(columns=cols)

# Populate the DataFrame
symbol_index = 100
df_num = 58
for i in range(0, len(symbols)):
  try:
    df = append_stock_stats(df, symbols[i], i)
  except:
    continue


  # In case we've used up our allotted requests per whatever,
  # we wait, then delete the last row and try again
  wait_time = 0
  while pd.isna(df['Market Cap'].iloc[-1]):
    time.sleep(90 + wait_time)
    wait_time += 1
    df.drop(df.tail(1).index, inplace=True)
    try:
      df = append_stock_stats(df, symbols[i], i)
    except:
      continue

  print(f'Row {i} fetched, Current stock: {symbols[i]}')

  if i % 100 == 0:
    !ls

  if i % 99 == 0 and i > 0:
    filename = f'df{df_num}.csv'
    df_num += 1
    df.to_csv(filename)
    df = pd.DataFrame(columns=cols)
    print(f'stocks {i-100} through {i} written')
    !ls


  # # Add a random delay because I am unreasonably cautious about getting\
  # # banned from yahoo finance
  delay = random.random() * 0.5 + 0.5
  time.sleep(delay)

df.head()

In [ ]:
'''
Get the stock metrics csv and clean it up a little. It will be cleaned more after
the stock prices have been added
'''

copy = pd.read_csv('df.csv', index_col=0)

for i in range(3, 67):
  df = pd.read_csv(f'df{i}.csv', index_col=0)
  copy = pd.concat([copy, df], axis=0, join='outer')
  print(copy.shape)

df = pd.read_csv('1585through1588.csv')
copy = pd.concat([copy, df], axis=0, join='outer')
df = pd.read_csv('496through499.csv')
copy = pd.concat([copy, df], axis=0, join='outer')

copy.reset_index()
copy.drop(columns=['Unnamed: 0', 'Unnamed: 11'])
copy.to_csv('df.csv')

In [ ]:
subprocess.run(["npm install dukascopy-node --save"])

FileNotFoundError: [Errno 2] No such file or directory: 'npm install dukascopy-node --save'

In [ ]:
dukascopy_symbols = []

for symbol in symbols:
  dukascopy_symbols.append(symbol.lower() + 'ususd')

for symbol in dukascopy_symbols:
  !npx dukascopy-node -i symbol -from 2023-06-01 -to 2024-06-01 -t mn1 -f csv --date-format "YYYY-MM-DD HH:mm:ss"

npx: installed 31 in 3.014s
Search config invalid:
 > The 'instrument' field does not match any of the allowed values.

npx: installed 31 in 2.608s
Search config invalid:
 > The 'instrument' field does not match any of the allowed values.

npx: installed 31 in 1.855s
Search config invalid:
 > The 'instrument' field does not match any of the allowed values.

^C
npx: installed 31 in 2.472s
Search config invalid:
 > The 'instrument' field does not match any of the allowed values.

^C
^C
^C
